## Import Libraries

In [10]:
#!/usr/bin/env python
# coding: utf-8
import os
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

## Import files

In [11]:
all_events = pd.read_csv(r"./data/2020-21/Cleaned_data/all_events.csv", parse_dates= ["deadline_time"])
all_players = pd.read_csv(r"./data/2020-21/Cleaned_data/all_players.csv", parse_dates= ["news_added"]) 


## Functions

In [12]:
#### Connect to FPL API
def get(url):
    response = requests.get(url)
    return json.loads(response.content)

## My Team

In [13]:
#### Get my fantasy team lineup
now = datetime.now()
my_team_id = '1341730'
current_gw = all_events[all_events['deadline_time'] < now]['gameweek_id'].max()

my_team_url = 'https://fantasy.premierleague.com/api/entry/{}/event/{}/picks/'.format(my_team_id,str(current_gw))

my_team = get(my_team_url)['picks']
my_team = pd.DataFrame(my_team)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_start', 'position_id', 'ep_next', 'ep_this',
       'event_points', 'first_name', 'player_id', 'player_news', 'news_added',
       'player_price', 'photo', 'second_name', 'selected_by_percent',
       'player_status', 'team_id', 'team_code_id', 'total_points',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'value_form', 'value_season', 'web_name',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank', 'creativity_rank_type', 'threat_rank',
       'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type'],
      dtype='object')

In [6]:
all_players = all_players[['chance_of_playing_next_round', 'chance_of_playing_this_round', 'position_id', 'ep_next','ep_this', 'player_id', 'player_news', 'news_added', 'player_price', 'selected_by_percent', 'player_status','team_id', 'web_name']]

In [7]:
my_team = my_team.merge(all_players, how = 'left', left_on="element", right_on="player_id")
my_team.drop(["element","position"], axis = 1, inplace = True)

In [8]:
my_team.sort_values(by=['position_id', 'selected_by_percent'])


,multiplier,is_captain,is_vice_captain,chance_of_playing_next_round,chance_of_playing_this_round,position_id,ep_next,ep_this,player_id,player_news,news_added,player_price,selected_by_percent,player_status,team_id,web_name
11,0,False,False,NaN,NaN,1,0.5,0.5,641,NaN,NaT,4.0,0.0,a,5,Jensen
0,1,False,False,100.0,100.0,1,5.7,5.1,93,NaN,2019-07-31 00:00:11.182051,5.2,23.1,a,5,Pope
14,0,False,False,100.0,100.0,2,0.5,0.5,583,NaN,2020-06-15 16:00:16.549558,3.9,1.3,a,17,Tanganga
3,1,False,False,NaN,NaN,2,5.5,4.9,331,NaN,NaT,5.3,5.0,a,17,Alderweireld
13,0,False,False,100.0,100.0,2,5.8,5.1,295,NaN,2020-06-21 14:30:21.128190,4.7,7.1,a,15,Egan
2,1,False,False,100.0,100.0,2,3.8,3.5,141,NaN,2020-02-23 19:00:12.788850,5.9,13.4,a,8,Digne
1,1,False,False,100.0,100.0,2,4.7,4.3,182,NaN,2019-10-22 15:00:19.720774,7.7,39.1,a,10,Alexander-Arnold
12,0,False,False,100.0,100.0,3,5.7,5.8,465,NaN,2019-08-31 16:30:09.941826,5.2,1.8,a,2,Trézéguet
4,1,False,True,100.0,100.0,3,7.0,6.6,219,NaN,2020-02-19 22:00:17.062940,7.5,9.1,a,11,David Silva
7,1,False,False,100.0,100.0,3,4.5,5.4,217,NaN,2020-01-27 16:30:23.592705,8.6,10.1,a,11,Mahrez


In [9]:
my_team.to_csv (r'./data/my_team_df.csv', index = False, header=True)